In [3]:
!pip install selenium
!pip install msedge-selenium-tools
!pip install bs4

  Using cached selenium-4.7.2-py3-none-any.whl (6.3 MB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached exceptiongroup-1.0.4-py3-none-any.whl (14 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


  Using cached msedge_selenium_tools-3.141.4-py3-none-any.whl (15 kB)
  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.7.2
    Uninstalling selenium-4.7.2:
      Successfully uninstalled selenium-4.7.2
  Using cached bs4-0.0.1-py3-none-any.whl


In [1]:
!pip install chromedriver_binary==109.0.5414.25

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for chromedriver_binary: filename=chromedriver_binary-109.0.5414.25.0-py3-none-any.whl size=7077909 sha256=8edec9633d8e7364219b698d58342fe674eec4f90127c125003aa46a394ff0da
  Stored in directory: c:\users\samar\appdata\local\pip\cache\wheels\c4\87\2b\ed76610d8039ef7f1bb1d4767327152091ebc5e73337586809
Successfully built chromedriver_binary


In [8]:
!pip install webdriver_manager

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)


In [1]:
from selenium import webdriver
import chromedriver_binary
import csv

In [65]:
from webdriver_manager.firefox import GeckoDriverManager
path=GeckoDriverManager().install()

# path='C:\\Users\\samar\\.wdm\\drivers\\geckodriver\\win64\\0.32\\geckodriver.exe'

driver = webdriver.Firefox(executable_path = path)

In [66]:
url= 'https://www.amazon.in/'
driver.get(url)

In [68]:
driver.get(url)

In [67]:
def my_url(keyword):
      temp = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss_1' # a template url
      keyword = keyword.replace(' ', '+')
      return temp.format(keyword)

In [71]:
url = my_url('smart watch')
driver.get(url)

In [72]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source,'html.parser')

In [74]:
soup_results=soup.find_all('div',{'data-component-type':'s-search-result'})

In [97]:
obj=soup_results[0]

In [98]:
atag = obj.h2.a #create the h2 tag variable

In [99]:
des = atag.text.strip()

In [100]:
des #Name of first block

'Noise ColorFit Pulse Grand Smart Watch with 1.69"(4.29cm) HD Display, 60 Sports Modes, 150 Watch Faces, Fast Charge, Spo2, Stress, Sleep, Heart Rate Monitoring & IP68 Waterproof (Jet Black)'

In [101]:
#create a generic url
url='https://www.amazon.in/'+atag.get('href')

In [102]:
atag.get('href')

'/sspa/click?ie=UTF8&spc=MTo4ODg5NDEzNDUzOTk5MjY3OjE2NzEwMTE0MTE6c3BfYXRmOjIwMDQwOTQxNTUwOTk4OjowOjo&url=%2FNoise-ColorFit-Display-Monitoring-Smartwatches%2Fdp%2FB09NVPSCQT%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dsmart%2Bwatch%26qid%3D1671011411%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1'

# Price

In [103]:
parent=obj.find('span','a-price')

price=parent.find('span','a-offscreen').text

price

'₹1,599'

# Ratings

In [104]:
rate=obj.find('span','a-icon-alt').text
rate

'4.0 out of 5 stars'

# Total Ratings

In [106]:
counts_review=obj.find('span',{'class':'a-size-base s-underline-text'}).text
counts_review

'28,598'

# Generic Fuction

In [171]:
from selenium import webdriver
import chromedriver_binary
from bs4 import BeautifulSoup
import csv


def my_url(keyword):
    temp = 'https://www.amazon.in/s?k={}'  #s?k=watch&page=7
    
    keyword = keyword.replace(' ', '+')
    url = temp.format(keyword)
    
    url += '&page={}'
    
    return url

def extract_record(obj):
    atag = obj.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    #it is possible that some items on amazom.com might not be having one of the items we are looking for(e.g. some items might not be having ratings or price), we will be getting error if we dont take care of that. We will therefore add some error handlers
    #if there are no price,probably the item is out of stock or not available, then we will ignore the item, but if there are no reviews yet, it's fine, we will still want to extract the item.
    try:
        parent=obj.find('span','a-price')
        price=parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        rate=obj.find('span','a-icon-alt').text
        counts_review = obj.find('span', {'class': 'a-size-base s-underline-text'}).text
    except AttributeError:
        rate = ''
        counts_review = ''
    
    image = obj.find('img', {'class': 's-image'}).get('src') 

    result = (description, price, rate, counts_review, url,image)
    return result






'''Run Main Program Routine'''
def main(keyword):
    driver = webdriver.Firefox(executable_path='C:\\Users\\samar\\.wdm\\drivers\\geckodriver\\win64\\0.32\\geckodriver.exe')

    
    records = [] 
    url =my_url(keyword)
    
    #SEPERATELY OPENING FIRST PAGE TO GET MAXIMUM AVAILABLE PAGES
    driver.get(url.format(1))
    soup =BeautifulSoup(driver.page_source, 'html.parser')
    possible_pages=int(soup.find('span',class_='s-pagination-item s-pagination-disabled').text)
    print(possible_pages)
    
    for page in range(1,possible_pages+1):
        driver.get(url.format(page))
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        results=soup.find_all('div',{'data-component-type':'s-search-result'})

        
        for item in results:
            record = extract_record(item) 
            if record: 
                records.append(record) 
                
    
        with open('Results.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Description', 'Price', 'Rating', 'Reviews Count', 'URL','Image link'])
            writer.writerows(records)

In [176]:
main('samsung m13')

8


In [177]:
import pandas as pd
df=pd.read_csv('Results.csv')
df

,Description,Price,Rating,Reviews Count,URL,Image link
0,"Samsung Galaxy M13 (Aqua Green, 4GB, 64GB Stor...","₹9,699",4.0 out of 5 stars,"16,878",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,https://m.media-amazon.com/images/W/WEBP_40237...
1,"Samsung Galaxy M13 (Aqua Green, 4GB, 64GB Stor...","₹9,699",4.0 out of 5 stars,"16,878",https://www.amazon.in/Samsung-Galaxy-Storage-6...,https://m.media-amazon.com/images/W/WEBP_40237...
2,"Samsung Galaxy M13 (Aqua Green, 6GB, 128GB Sto...","₹11,699",4.0 out of 5 stars,"16,878",https://www.amazon.in/Samsung-Galaxy-Storage-6...,https://m.media-amazon.com/images/W/WEBP_40237...
3,"Samsung Galaxy M13 (Midnight Blue, 4GB, 64GB S...","₹9,699",4.0 out of 5 stars,"16,878",https://www.amazon.in/Samsung-Midnight-Storage...,https://m.media-amazon.com/images/W/WEBP_40237...
4,"Samsung Galaxy M13 5G (Midnight Blue, 4GB, 64G...","₹13,999",4.0 out of 5 stars,"16,878",https://www.amazon.in/Samsung-Midnight-Storage...,https://m.media-amazon.com/images/W/WEBP_40237...
...,...,...,...,...,...,...
117,"Samsung Galaxy M53 5G (Emerald Brown, 6GB, 128...","₹26,499",3.9 out of 5 stars,"8,335",https://www.amazon.in/Samsung-Emerald-Storage-...,https://m.media-amazon.com/images/W/WEBP_40237...
118,"Samsung Galaxy S20 FE 5G (Cloud Lavender, 8GB ...","₹37,990",4.2 out of 5 stars,"26,896",https://www.amazon.in/Samsung-Galaxy-Cloud-Lav...,https://m.media-amazon.com/images/W/WEBP_40237...
119,"Samsung Galaxy S22 Ultra 5G (Phantom Black, 12...","₹1,12,998",4.4 out of 5 stars,"1,388",https://www.amazon.in/Samsung-Galaxy-Phantom-S...,https://m.media-amazon.com/images/W/WEBP_40237...
120,"Samsung Galaxy S22 Ultra 5G (Burgundy, 12GB, 5...","₹1,18,999",4.4 out of 5 stars,"1,388",https://www.amazon.in/Samsung-Burgundy-Storage...,https://m.media-amazon.com/images/W/WEBP_40237...
